
# CS436 Project: 3D Scene Reconstruction
## Week 3: Range-Based Multi-View Reconstruction

This version extends Week 2's two-view pipeline to process consecutive image ranges (4-6 images per group). Instead of pairwise reconstruction, we define ranges like [(0,5), (6,9), (11,21)] and perform incremental SfM within each range using the stable Week 2 approach.



**Pipeline**
- Load images from `../data` (optionally downscale) and compute intrinsics from image size.
- Define image ranges as an array of tuples: `IMAGE_RANGES = [(0,5), (6,9), (11,21)]`
- For each range: perform incremental SfM using Week 2's two-view approach:
  - Build consecutive pairs within the range (e.g., 0-1, 1-2, 2-3, 3-4, 4-5 for range (0,5))
  - Dense SIFT+ORB matching, Essential matrix (tight RANSAC), recover pose, triangulate
  - Chain poses incrementally and merge points from all pairs in the range
  - Save as `ply_ranges/range_0_to_5.ply`
- Optional: Merge all range clouds and visualize with Open3D


In [1]:

import os, sys, gc
import numpy as np
import cv2

PROJECT_ROOT = os.path.abspath(os.path.join('..'))
SRC_DIR = os.path.join(PROJECT_ROOT, 'src')
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from utils import (
    load_images_from_folder,
    get_dense_combined_matches,
    sample_colors,
    save_colored_point_cloud_to_ply,
    save_point_cloud_to_ply,
    set_global_memory_mode,
    filter_points,
)
from reconstruction import (
    get_intrinsic_matrix,
    calculate_essential_matrix_from_points,
    recover_camera_pose,
    triangulate_points,
    reconstruct_pair,
    reconstruct_range,
)

set_global_memory_mode()
print('Imports ready; memory mode on.')


Imports ready; memory mode on.


## Configuration

In [2]:
DATA_DIR = '../data'
RANGE_OUTPUT_DIR = 'ply_ranges'   # keep outputs inside notebooks/
MAX_VIEWS = None                  # set int to limit number of images (e.g., 20)
DOWNSCALE_MAX_SIDE = 1280         # None to disable; tradeoff: more detail vs RAM

# Define image ranges: each tuple (start, end) inclusive
IMAGE_RANGES = [(0, 6), (7, 15), (16, 22), (23, 32)]  # Modify this for your dataset

# Reconstruction tunables
RANSAC_THRESH = 1.5             # px threshold for Essential matrix
MIN_COMBINED_MATCHES = 40
MIN_INLIERS = 25
MIN_CHEIRALITY = 15
MAX_DIST_PERCENTILE = 98        # drop extreme far points
VOXEL_SIZE = 0.0                # set >0 (e.g., 0.01) to downsample merged cloud
OUTLIER_STD = 2.0               # statistical outlier removal std ratio

os.makedirs(RANGE_OUTPUT_DIR, exist_ok=True)
print(f'Range outputs will be saved under {RANGE_OUTPUT_DIR}/')
print(f'Configured image ranges: {IMAGE_RANGES}')


Range outputs will be saved under ply_ranges/
Configured image ranges: [(0, 6), (7, 15), (16, 22), (23, 32)]


## Load images

In [3]:
images = load_images_from_folder(DATA_DIR, target_width=DOWNSCALE_MAX_SIDE or 1024)
if MAX_VIEWS is not None:
    images = images[:MAX_VIEWS]
print(f'Total images loaded: {len(images)}')
assert len(images) >= 2, 'Need at least two images to reconstruct.'

K = get_intrinsic_matrix(images[0].shape)
print('Intrinsic matrix (approx):')
print(K)


Successfully loaded 33 images from ../data
Images resized to max width: 1280px
Total images loaded: 33
Intrinsic matrix (approx):
[[1280.    0.  640.]
 [   0. 1280.  853.]
 [   0.    0.    1.]]


## Validate ranges

In [4]:

num_images = len(images)
print(f'Validating {len(IMAGE_RANGES)} ranges against {num_images} images...')

valid_ranges = []
for start, end in IMAGE_RANGES:
    if start < 0 or end >= num_images:
        print(f'  Warning: Range ({start},{end}) is out of bounds [0,{num_images-1}]. Skipping.')
        continue
    if end - start < 1:
        print(f'  Warning: Range ({start},{end}) has fewer than 2 images. Skipping.')
        continue
    valid_ranges.append((start, end))
    print(f'  ✓ Range ({start},{end}): {end-start+1} images')

if not valid_ranges:
    raise ValueError('No valid ranges found. Check IMAGE_RANGES configuration.')

IMAGE_RANGES = valid_ranges
print(f'\nProceeding with {len(IMAGE_RANGES)} valid ranges.')


Validating 4 ranges against 33 images...
  ✓ Range (0,6): 7 images
  ✓ Range (7,15): 9 images
  ✓ Range (16,22): 7 images
  ✓ Range (23,32): 10 images

Proceeding with 4 valid ranges.


## Run range-based reconstruction and save per-range PLYs

In [5]:

# Run range-based reconstruction and save per-range PLYs
range_summaries = []

for start_idx, end_idx in IMAGE_RANGES:
    # Call reconstruct_range from src/reconstruction.py
    merged_pts, merged_cols = reconstruct_range(
        (start_idx, end_idx), 
        images, 
        K,
        min_matches=MIN_COMBINED_MATCHES,
        min_inliers=MIN_INLIERS,
        min_cheirality=MIN_CHEIRALITY,
        ransac_thresh=RANSAC_THRESH,
        max_dist_percentile=MAX_DIST_PERCENTILE
    )
    
    if merged_pts is None:
        print(f'Range ({start_idx}, {end_idx}): Reconstruction failed.\n')
        range_summaries.append(((start_idx, end_idx), 0))
        continue
    
    # Save range point cloud
    range_filename = f'range_{start_idx}_to_{end_idx}.ply'
    range_path = os.path.join(RANGE_OUTPUT_DIR, range_filename)
    
    try:
        if merged_cols is not None:
            save_colored_point_cloud_to_ply(merged_pts, merged_cols, range_path)
        else:
            save_point_cloud_to_ply(merged_pts, range_path)
        print(f'✅ Saved {range_path} ({len(merged_pts)} points)\n')
        range_summaries.append(((start_idx, end_idx), len(merged_pts)))
    except Exception as e:
        print(f'❌ Failed to save {range_path}: {e}\n')
        range_summaries.append(((start_idx, end_idx), 0))

print('\n=== Range Reconstruction Summary ===')
for (start, end), num_pts in range_summaries:
    status = '✓' if num_pts > 0 else '✗'
    print(f'  {status} Range ({start},{end}): {num_pts} points')


\n=== Reconstructing Range (0, 6): 7 images ===
  Pair 0 -> 1
    Matches: SIFT=1791 ORB=1869 Combined=3660
    RANSAC inliers: 2150
    Cheirality passed: 2149
    Pose refined via PnP (inliers: 2063)
    Reconstructed points: 2063
  Set pair 0-1 as reference frame.
  Pair 1 -> 2
    Matches: SIFT=1791 ORB=1869 Combined=3660
    RANSAC inliers: 2150
    Cheirality passed: 2149
    Pose refined via PnP (inliers: 2063)
    Reconstructed points: 2063
  Set pair 0-1 as reference frame.
  Pair 1 -> 2
    Matches: SIFT=1566 ORB=1393 Combined=2959
    RANSAC inliers: 1993
    Cheirality passed: 1993
    Pose refined via PnP (inliers: 1913)
    Reconstructed points: 1913
  Chained pair 1-2 to global frame.
  Pair 2 -> 3
    Matches: SIFT=1566 ORB=1393 Combined=2959
    RANSAC inliers: 1993
    Cheirality passed: 1993
    Pose refined via PnP (inliers: 1913)
    Reconstructed points: 1913
  Chained pair 1-2 to global frame.
  Pair 2 -> 3
    Matches: SIFT=1410 ORB=1165 Combined=2575
    RANSAC

## Merge all range clouds into one unified PLY

In [6]:

MERGED_PLY = os.path.join(RANGE_OUTPUT_DIR, 'reconstruction_all_ranges_merged.ply')

try:
    import open3d as o3d
    HAS_O3D = True
except ImportError:
    HAS_O3D = False
    print('⚠️  Open3D not installed. Merging will be skipped.')
    print('   Install with: pip install open3d')

if HAS_O3D:
    import glob
    
    # Find all range PLYs
    ply_files = sorted([f for f in glob.glob(os.path.join(RANGE_OUTPUT_DIR, 'range_*.ply'))])
    
    if not ply_files:
        print('No range PLYs found to merge.')
    else:
        print(f'Merging {len(ply_files)} range clouds...')
        pts_list, col_list = [], []
        
        for f in ply_files:
            pcd = o3d.io.read_point_cloud(f)
            pts = np.asarray(pcd.points)
            if pts.size == 0:
                print(f'  Skipped {os.path.basename(f)} (empty)')
                continue
            pts_list.append(pts)
            if pcd.has_colors():
                col_list.append(np.asarray(pcd.colors))
            else:
                col_list.append(np.full((pts.shape[0], 3), 0.5, dtype=np.float64))
            print(f'  Loaded {os.path.basename(f)}: {len(pts)} points')
        
        if not pts_list:
            print('No valid point clouds to merge.')
        else:
            merged_pts = np.vstack(pts_list)
            merged_cols = np.vstack(col_list)
            
            pcd_merged = o3d.geometry.PointCloud()
            pcd_merged.points = o3d.utility.Vector3dVector(merged_pts)
            pcd_merged.colors = o3d.utility.Vector3dVector(merged_cols)
            
            # Optional: voxel downsampling
            if VOXEL_SIZE and VOXEL_SIZE > 0:
                print(f'Applying voxel downsampling (voxel_size={VOXEL_SIZE})...')
                pcd_merged = pcd_merged.voxel_down_sample(VOXEL_SIZE)
                print(f'  After downsampling: {len(pcd_merged.points)} points')
            
            # Optional: statistical outlier removal
            if OUTLIER_STD and OUTLIER_STD > 0:
                print(f'Removing statistical outliers (std_ratio={OUTLIER_STD})...')
                cl, ind = pcd_merged.remove_statistical_outlier(nb_neighbors=40, std_ratio=OUTLIER_STD)
                pcd_merged = cl
                print(f'  After outlier removal: {len(pcd_merged.points)} points')
            
            o3d.io.write_point_cloud(MERGED_PLY, pcd_merged, write_ascii=True)
            print(f'\n✅ Saved merged cloud: {MERGED_PLY}')
            print(f'   Total points: {len(pcd_merged.points)}')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Merging 4 range clouds...
  Loaded range_0_to_6.ply: 9017 points
  Loaded range_16_to_22.ply: 8617 points
  Loaded range_23_to_32.ply: 11143 points
  Loaded range_7_to_15.ply: 13447 points
Removing statistical outliers (std_ratio=2.0)...
  After outlier removal: 40312 points

✅ Saved merged cloud: ply_ranges\reconstruction_all_ranges_merged.ply
   Total points: 40312

✅ Saved merged cloud: ply_ranges\reconstruction_all_ranges_merged.ply
   Total points: 40312


In [7]:
# Optional: preview merged cloud
try:
    import open3d as o3d
    if os.path.exists(MERGED_PLY):
        pcd = o3d.io.read_point_cloud(MERGED_PLY)
        if len(np.asarray(pcd.points)) == 0:
            print('Merged cloud is empty.')
        else:
            bbox = pcd.get_axis_aligned_bounding_box()
            bbox.color = (1, 0.6, 0)
            origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
            print(f'Launching Open3D viewer with {len(pcd.points)} points...')
            print('🎮 Controls:')
            print('   • Left-click + drag = Rotate')
            print('   • Right-click + drag = Pan')
            print('   • Scroll = Zoom')
            print('   • Press H = Show all shortcuts')
            o3d.visualization.draw_geometries([pcd, bbox, origin], 
                                             window_name="Range-Based Reconstruction - Merged Cloud",
                                             width=1400, height=1000)
    else:
        print(f'Merged PLY not found at {MERGED_PLY}. Run merge cell first.')
except ImportError:
    print('Open3D not installed. Skipping visualization.')


Launching Open3D viewer with 40312 points...
🎮 Controls:
   • Left-click + drag = Rotate
   • Right-click + drag = Pan
   • Scroll = Zoom
   • Press H = Show all shortcuts


## Optional: ICP-Based Refinement of Range Clouds

In [8]:

print("\n=== ICP-Based Alignment for Range Clouds ===")
try:
    import open3d as o3d
    import glob
    
    # Find all range PLYs in order
    ply_files = sorted([f for f in glob.glob(os.path.join(RANGE_OUTPUT_DIR, 'range_*.ply'))])
    
    if not ply_files:
        print("No range PLYs found. Run the reconstruction cell first.")
    else:
        print(f"Found {len(ply_files)} range clouds to align.")
        
        # Load all clouds
        clouds = []
        for f in ply_files:
            pcd = o3d.io.read_point_cloud(f)
            if len(np.asarray(pcd.points)) > 0:
                clouds.append(pcd)
                print(f"  Loaded {os.path.basename(f)}: {len(pcd.points)} points")
            else:
                print(f"  Skipped {os.path.basename(f)} (empty)")
        
        if len(clouds) < 2:
            print("Not enough non-empty clouds to align (need at least 2).")
        else:
            # Chain ICP: align each cloud to the previous one
            print("\nStarting chained ICP alignment...")
            transformations = [np.eye(4)]  # First cloud is reference
            
            for i in range(1, len(clouds)):
                src = clouds[i]
                ref = clouds[i-1]
                
                # ICP registration
                reg_result = o3d.pipelines.registration.registration_icp(
                    src, ref,
                    max_correspondence_distance=1.0,  # Adjust based on scale
                    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(
                        relative_fitness=1e-6,
                        relative_rmse=1e-6,
                        max_iteration=50
                    )
                )
                
                # Apply transformation to current cloud
                src.transform(reg_result.transformation)
                transformations.append(reg_result.transformation)
                
                fitness = reg_result.fitness
                rmse = reg_result.inlier_rmse
                print(f"  Range {i-1}→{i}: fitness={fitness:.4f}, RMSE={rmse:.6f}")
            
            # Merge all aligned clouds
            print("\nMerging ICP-aligned clouds...")
            merged_icp = o3d.geometry.PointCloud()
            for cloud in clouds:
                merged_icp += cloud
            
            # Optional: downsample
            if VOXEL_SIZE and VOXEL_SIZE > 0:
                merged_icp = merged_icp.voxel_down_sample(VOXEL_SIZE)
                print(f"Downsampled to {len(merged_icp.points)} points (voxel_size={VOXEL_SIZE}).")
            
            # Optional: outlier removal
            if OUTLIER_STD and OUTLIER_STD > 0:
                cl, ind = merged_icp.remove_statistical_outlier(nb_neighbors=40, std_ratio=OUTLIER_STD)
                merged_icp = cl
                print(f"Removed outliers, {len(merged_icp.points)} points remain.")
            
            # Save refined cloud
            ICP_MERGED_PLY = os.path.join(RANGE_OUTPUT_DIR, 'reconstruction_icp_refined.ply')
            o3d.io.write_point_cloud(ICP_MERGED_PLY, merged_icp, write_ascii=True)
            print(f"\n✅ Saved ICP-refined merged cloud: {ICP_MERGED_PLY}")
            print(f"   Total points: {len(merged_icp.points)}")
            
            # Visualize result
            print("\nLaunching Open3D viewer for ICP-refined cloud...")
            bbox = merged_icp.get_axis_aligned_bounding_box()
            bbox.color = (1, 0.6, 0)
            origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
            o3d.visualization.draw_geometries([merged_icp, bbox, origin], 
                                             window_name="ICP-Refined Range Clouds",
                                             width=1400, height=1000)
            
except ImportError:
    print("⚠️  Open3D not installed. Skipping ICP refinement.")
    print("   Install with: pip install open3d")
except Exception as e:
    print(f"⚠️  ICP alignment error: {e}")



=== ICP-Based Alignment for Range Clouds ===
Found 4 range clouds to align.
  Loaded range_0_to_6.ply: 9017 points
  Loaded range_16_to_22.ply: 8617 points
  Loaded range_23_to_32.ply: 11143 points
  Loaded range_7_to_15.ply: 13447 points

Starting chained ICP alignment...
  Range 0→1: fitness=0.5169, RMSE=0.557304
  Range 1→2: fitness=0.6745, RMSE=0.479104
  Range 1→2: fitness=0.6745, RMSE=0.479104
  Range 2→3: fitness=0.9382, RMSE=0.385540

Merging ICP-aligned clouds...
Removed outliers, 40490 points remain.
  Range 2→3: fitness=0.9382, RMSE=0.385540

Merging ICP-aligned clouds...
Removed outliers, 40490 points remain.

✅ Saved ICP-refined merged cloud: ply_ranges\reconstruction_icp_refined.ply
   Total points: 40490

Launching Open3D viewer for ICP-refined cloud...

✅ Saved ICP-refined merged cloud: ply_ranges\reconstruction_icp_refined.ply
   Total points: 40490

Launching Open3D viewer for ICP-refined cloud...
